In [9]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate,PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator
from langchain_core.runnables import ConfigurableField
from langchain.runnables.hub import HubRunnable
from langchain_anthropic import ChatAnthropic
from langchain_community.utils.math import cosine_similarity

In [10]:
load_dotenv(find_dotenv("../.env"))

True

In [11]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmGPT=ChatOpenAI(model="gpt-4O-mini")

<h3>Using Runnable Lambda</h3>

In [14]:
prompt=PromptTemplate.from_template(template="""
    Given the user question below, classify it as either being about `Langchain`,`Anthropic`, or `Other`

    Do not respond with more than one word.

    <question>
    {question}
    </question>

    Classification:
""")

In [4]:
chain=prompt|llmGemini|StrOutputParser()

In [5]:
chain.invoke(input={"question":"What is Claude LLM?"})

'Anthropic'

In [6]:
langchainChain=PromptTemplate.from_template(
    template="""
        You are an expert in langchain.\
        Always answer questions starting with "As Prof. Langchain told me". \
        Respond to the following question:

        Question: {question}
        Answer:
    """
)| llmGemini|StrOutputParser()

In [7]:
anthropicChain=PromptTemplate.from_template(
    template="""
        You are an expert in anthropic.\
        Always answer questions starting with "As Prof. Anthropic told me". \
        Respond to the following question:

        Question: {question}
        Answer:
    """
)| llmGemini|StrOutputParser()

In [8]:
generalChain=PromptTemplate.from_template(
    template="""
        Respond to the following question:

        Question: {question}
        Answer:
    """
)| llmGemini|StrOutputParser()

In [15]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropicChain
    elif "langchain" in info["topic"].lower():
        return langchainChain
    else:
        return generalChain

In [16]:
finalChain=RunnableParallel(
    {"topic":chain,"question":RunnablePassthrough()}
)|RunnableLambda(func=route)|StrOutputParser()

In [17]:
finalChain.invoke(input="What is Claude?")

"As Prof. Anthropic told me, Claude is a family of large language models created by Anthropic. It's designed to be helpful, harmless, and honest, aiming to be a useful AI assistant. Claude is known for its strong reasoning abilities, its ability to generate creative content, and its commitment to safety and ethical considerations in AI. Think of it as a powerful AI that's been carefully trained to be a responsible and beneficial tool."

In [18]:
finalChain.invoke(input="What is ChatPromptTemplate?")

'As Prof. Langchain told me, a `ChatPromptTemplate` in Langchain is a way to structure and define prompts specifically for chat models. It\'s designed to handle the multi-turn conversational nature of chat models, where you have a sequence of messages (e.g., system message, human message, AI message) rather than just a single prompt string.\n\nHere\'s a breakdown of what it is and why it\'s useful:\n\n*   **Purpose:** To create structured prompts for chat models, allowing you to define different roles (system, human, AI) and their corresponding message content.\n\n*   **Key Features:**\n\n    *   **Message Templates:** It\'s built from a sequence of `BaseMessagePromptTemplate` objects. These templates define the role (e.g., `SystemMessagePromptTemplate`, `HumanMessagePromptTemplate`, `AIMessagePromptTemplate`) and the content of each message.\n    *   **Variables:**  Message templates can include variables that are dynamically filled in at runtime using user-provided data. This allows 

In [19]:
print(finalChain.invoke(input="Who was Eklavya?"))

Eklavya was a remarkable character from the Hindu epic, the Mahabharata. He was a young tribal prince known for his exceptional archery skills and unwavering devotion to his guru, Dronacharya. Although Dronacharya refused to accept him as a formal student due to his caste, Eklavya practiced diligently on his own, using a clay statue of Dronacharya as his guide. He became a more skilled archer than Arjuna, Dronacharya's favorite and most prized student. To uphold his promise to Arjuna that he would be the greatest archer, Dronacharya asked Eklavya for his right thumb as guru dakshina (teacher's fee). Eklavya, without hesitation, sacrificed his thumb, demonstrating his ultimate respect and devotion to his guru. This act crippled his archery abilities, but cemented his place in the epic as a symbol of dedication, self-learning, and the complexities of social hierarchy in ancient India.


<h3> Routing by Symantic Similarity</h3>

In [20]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

In [21]:
math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

In [22]:
embeddings=SentenceTransformerEmbeddings()

C:\Users\MSI\AppData\Local\Temp\ipykernel_1456\481481593.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=SentenceTransformerEmbeddings()
C:\Users\MSI\AppData\Local\Temp\ipykernel_1456\481481593.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=SentenceTransformerEmbeddings()
c:\Users\MSI\anaconda3\envs\lcnenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedoc

In [23]:
promptTemplates=[physics_template,math_template]
promptEmbeddings=embeddings.embed_documents(texts=promptTemplates)

In [27]:
len(promptEmbeddings),len(promptEmbeddings[0])

(2, 768)

In [33]:
# Illustrations
questions=["What's a Black Hole?","What's Integral Calculus?"]

In [36]:
cosine_similarity(X=embeddings.embed_documents(texts=questions),Y=promptEmbeddings)

array([[0.25714647, 0.18505651],
       [0.242622  , 0.2775209 ]])

In [37]:
cosine_similarity(X=embeddings.embed_documents(texts=questions),Y=promptEmbeddings).argmax(axis=0)

array([0, 1])

In [39]:
def promptRouter(input) -> ChatPromptTemplate:
    queryEmbedding=embeddings.embed_query(text=input['query'])
    similarity=cosine_similarity(X=[queryEmbedding],Y=promptEmbeddings)[0]
    mostSimilarTemplate=promptTemplates[similarity.argmax()]
    print("Using Math Template" if mostSimilarTemplate==math_template else "Using Physics Template")
    return PromptTemplate.from_template(template=mostSimilarTemplate)

In [40]:
chain={"query":RunnablePassthrough()}|RunnableLambda(func=promptRouter)|llmGemini|StrOutputParser()

In [41]:
chain.invoke(input="What's a Blackhole?")

Using Physics Template


'Ah, an excellent question!\n\nIn a nutshell, a black hole is a region in spacetime where gravity is so strong that nothing, not even light, can escape.\n\nHere\'s a breakdown:\n\n*   **Formation:** Black holes typically form from the remnants of massive stars that have exhausted their nuclear fuel. When these stars die, they collapse under their own gravity. If the star is massive enough, this collapse continues until all of its matter is crushed into an incredibly small space.\n\n*   **Event Horizon:** The "point of no return" around a black hole is called the event horizon. It\'s the boundary beyond which escape is impossible. Think of it like a waterfall: once you\'re over the edge, there\'s no going back.\n\n*   **Singularity:** At the center of a black hole, according to current theory, lies a singularity. This is a point of infinite density where the laws of physics as we know them break down.\n\n*   **Gravity:** The immense gravity of a black hole warps the space and time aroun

In [42]:
chain.invoke(input="What's Trigonometry?")

Using Math Template


"Okay, I can definitely break down the concept of trigonometry and explain it clearly. Here's how I'll approach it:\n\n1.  **Core Meaning/Etymology:** Start with the basic definition and where the word comes from.\n2.  **Fundamental Concepts:** Explain the core relationships between angles and sides of triangles, especially right triangles.  This includes the trigonometric functions (sine, cosine, tangent, etc.).\n3.  **Unit Circle:** Explain the unit circle and how it extends trigonometric functions beyond acute angles.\n4.  **Applications:** Briefly touch on *why* trigonometry is useful in various fields.\n5.  **Generalizations:** Mention how trigonometry extends beyond basic triangles to more abstract concepts.\n\nHere we go:\n\n**1. Core Meaning/Etymology:**\n\n*   Trigonometry, at its heart, is the study of the relationships between the angles and sides of triangles.\n*   The word itself comes from Greek:\n    *   *trigōnon* (triangle) + *metron* (measure)\n\n**2. Fundamental Conc

In [43]:
chain.get_graph().print_ascii()

 +----------------------+  
 | Parallel<query>Input |  
 +----------------------+  
             *             
             *             
             *             
      +-------------+      
      | Passthrough |      
      +-------------+      
             *             
             *             
             *             
     +--------------+      
     | promptRouter |      
     +--------------+      
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
